# The Path Forward
If the WASM slowdown is generically by 1/3.4, then our heuristics say we should aim the follow time on 16 cores for an x86 build:


In [1]:
TARGET_TIME = 60/3.4
TARGET_TIME 

17.647058823529413

Set up the basic utility class to compute and display the effects of our optimizations

In [2]:
from prettytable import PrettyTable
FIELD_NAMES = ["-", "total", "rels", "msm", "trace", "etc"]
NUM_RELATIONS = 10 
NUM_THREADS = 16

def normalize(d):
    total = sum(d.values())
    return {k: round(100*v/total) for (k, v) in d.items()}


class TableData:
    def __init__(self, percs, times):
        self.percs = percs
        self.times = times

    def to_table(self):
        result = PrettyTable()
        result.field_names = FIELD_NAMES

        percs = list(self.percs.values())
        times = list(self.times.values())
        result.add_row(["%"] + [round(sum(percs), 2)] + percs)
        result.add_row(["#(s)"] + [round(sum(times), 2)] + times)
        return result

    def update_perc(self, k, v):
        self.percs[k] = self.percs[k] * v
        self.percs = normalize(self.percs)

    def update_time(self, k, v):
        self.times[k] = round(self.times[k] * v, 2)

    def update(self, k, v):
        self.update_perc(k, v)
        self.update_time(k, v)

    def __str__(self):
        return self.to_table().__str__()

In [3]:
INITIAL_PERCS = {"rels": 20,   "msm": 30,   "trace": 35,    "etc": 15}
INITIAL_TIMES = {"rels": 34.6, "msm": 51.9, "trace": 60.55, "etc": 25.95}
T = TableData(INITIAL_PERCS, INITIAL_TIMES)
print(T)

+------+-------+------+------+-------+-------+
|  -   | total | rels | msm  | trace |  etc  |
+------+-------+------+------+-------+-------+
|  %   |  100  |  20  |  30  |   35  |   15  |
| #(s) | 173.0 | 34.6 | 51.9 | 60.55 | 25.95 |
+------+-------+------+------+-------+-------+


## Multithreaded witness construction

Our default assumption is multithreading over 16 cores. In this case, since there is a lot of stdlib and we will likely only multithread a few hot loops, we can be pessimistic and assume that we will only get an $8\times$ speedup.


In [4]:
REDUCTION_DUE_TO_MULTITHREADED_WITNESS_CONSTRUCTION = 1/8
T.update("trace", REDUCTION_DUE_TO_MULTITHREADED_WITNESS_CONSTRUCTION)
print(T)


+------+--------+------+------+-------+-------+
|  -   | total  | rels | msm  | trace |  etc  |
+------+--------+------+------+-------+-------+
|  %   |  100   |  29  |  43  |   6   |   22  |
| #(s) | 120.02 | 34.6 | 51.9 |  7.57 | 25.95 |
+------+--------+------+------+-------+-------+


## Using Protogalaxy

For now, let's just estimate the effect of Protogalaxy by saying that it achieves the following change in ratios: $\frac{\text{rels}}{\text{msm}} = \frac{40}{36}$. In that case we want to reduce the MSM time by $x$ such that $\frac{\text{rels}}{x\times \text{msm}} = \frac{40}{36}$.

In [5]:
PROTOGALAXY_EXPECTED_RELS_PERC = 40
PROTOGALAXY_EXPECTED_MSM_PERC = 36
REDUCTION_DUE_TO_PROTOGALAXY = (T.percs["rels"]/T.percs["msm"])/(40/36)
T.update("msm", REDUCTION_DUE_TO_PROTOGALAXY)
print(T)


+------+-------+------+------+-------+-------+
|  -   | total | rels | msm  | trace |  etc  |
+------+-------+------+------+-------+-------+
|  %   |   99  |  35  |  31  |   7   |   26  |
| #(s) | 99.62 | 34.6 | 31.5 |  7.57 | 25.95 |
+------+-------+------+------+-------+-------+


## Structuring the execution trace

By structuring the exeuction trace, we can reduce the time to execute the relations by a factor of $1/\text{NUM\_RELATIONS}$

In [6]:
REDUCTION_DUE_TO_TRACE_STRUCTURING = 1/NUM_RELATIONS
T.update("rels", REDUCTION_DUE_TO_TRACE_STRUCTURING)
print(T)


+------+-------+------+------+-------+-------+
|  -   | total | rels | msm  | trace |  etc  |
+------+-------+------+------+-------+-------+
|  %   |  100  |  5   |  46  |   10  |   39  |
| #(s) | 68.48 | 3.46 | 31.5 |  7.57 | 25.95 |
+------+-------+------+------+-------+-------+


## Improved multithreading

OMP did a better job by about 35% on the sizes we care about in terms of commitments. Assume that holds and we can get the corresponding reduction in x86.

In [7]:
REDUCTION_DUE_TO_OMP = 0.65
T.update("msm", REDUCTION_DUE_TO_OMP)
print(T)


+------+-------+------+-------+-------+-------+
|  -   | total | rels |  msm  | trace |  etc  |
+------+-------+------+-------+-------+-------+
|  %   |  100  |  6   |   36  |   12  |   46  |
| #(s) | 57.46 | 3.46 | 20.48 |  7.57 | 25.95 |
+------+-------+------+-------+-------+-------+


## Multithreading compute_full_honk_evaluations

Row now, 9% of Protogalaxy proving time is not multithreaded (according to xray...). A bulk of that is taken up by the non-multithreaded function `compute_full_honk_evaluations` which accounts for 3.75% of the total proving. Suppose we can mulitithread that effectively.

In [8]:
COMPUTE_EVALS_PERC = 3.75
SINGLY_THREADED_PERC = 9


REDUCTION_DUE_TO_MULTITHREADING_COMPUTE_EVALS = 1 - (COMPUTE_EVALS_PERC/SINGLY_THREADED_PERC) * (1-1/NUM_THREADS)
T.update("etc", REDUCTION_DUE_TO_MULTITHREADING_COMPUTE_EVALS)
print(T)


+------+-------+------+-------+-------+-------+
|  -   | total | rels |  msm  | trace |  etc  |
+------+-------+------+-------+-------+-------+
|  %   |  100  |  7   |   44  |   15  |   34  |
| #(s) | 47.32 | 3.46 | 20.48 |  7.57 | 15.81 |
+------+-------+------+-------+-------+-------+


## Better field arithemetic

Somebody has written BN254 field arithemtic in Nim and it is 30% faster.

In [9]:
REDUCTION_DUE_TO_USING_NIM = 0.7
T.update("msm", REDUCTION_DUE_TO_USING_NIM)
T.update("rels", REDUCTION_DUE_TO_USING_NIM)
print(T)


+------+-------+------+-------+-------+-------+
|  -   | total | rels |  msm  | trace |  etc  |
+------+-------+------+-------+-------+-------+
|  %   |  100  |  6   |   36  |   18  |   40  |
| #(s) | 40.14 | 2.42 | 14.34 |  7.57 | 15.81 |
+------+-------+------+-------+-------+-------+
